In [1]:
import xarray as xr
import numpy as np
from dask.distributed import Client
import pandas as pd
import dask

In [2]:
client = Client(n_workers=7, threads_per_worker=2, memory_limit='50GB')
client

distributed.diskutils - INFO - Found stale lock file and directory '/home/eouser/Documents/code/Windatlas/windatlas/anemos_data/dask-worker-space/worker-kku99eeg', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/eouser/Documents/code/Windatlas/windatlas/anemos_data/dask-worker-space/worker-ah03uzbu', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/eouser/Documents/code/Windatlas/windatlas/anemos_data/dask-worker-space/worker-_k5ex9wt', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/eouser/Documents/code/Windatlas/windatlas/anemos_data/dask-worker-space/worker-aa56w0qx', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/eouser/Documents/code/Windatlas/windatlas/anemos_data/dask-worker-space/worker-2uodr0h1', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/eouser/Documents/code/Windatlas/windatlas/anemos_data/dask-worker-

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 7
Total threads: 14,Total memory: 325.96 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33855,Workers: 7
Dashboard: http://127.0.0.1:8787/status,Total threads: 14
Started: Just now,Total memory: 325.96 GiB
Comm: tcp://10.0.0.42:37499,Total threads: 2
Dashboard: http://10.0.0.42:37623/status,Memory: 46.57 GiB
Nanny: tcp://127.0.0.1:43217,


In [3]:
def coor_transformation(coor:list,engine:str="pyproj")->list:
    lambert_proj_str = "+proj=lcc +lat_1=48.0 +lat_2=54.0 +lat_0=50.893 +lon_0=10.736 +a=6370000 +b=6370000 +nadgrids=null +no_defs"

    if engine == "pyproj":
        from pyproj import CRS, Transformer

        crsD3E5 = CRS.from_proj4(lambert_proj_str)
        crsGeo = CRS.from_epsg(4326)
        geo2altas = Transformer.from_crs(crsGeo, crsD3E5)

        x,y = geo2altas.transform(coor[0], coor[1])
        return[x,y]

    if engine == "gdal":
        from ogr import Geometry
        from osr import SpatialReference
        
        d3e5Prj = SpatialReference()
        d3e5Prj.ImportFromProj4(lambert_proj_str)
        geoPrj = SpatialReference()
        geoPrj.ImportFromEPSG(4326)

        point = Geometry(ogr.wkbPoint)
        point.AddPoint(coor[0], coor[1])
        point.AssignSpatialReference(geoPrj)    # tell the point what coordinates it's in
        point.TransformTo(d3e5Prj)              # project it to the out spatial reference
        return[point.GetX(),point.GetY()]

In [4]:
years = np.arange(2009,2019,1)
year = 2017
lon = 14.0
lat = 55.0
level = 113
[x_interp,y_interp] = coor_transformation(coor=[lat,lon], engine="pyproj")

# load the new dimensional lambda coordinates
path = "./lambert_projection/xy_lamber_projection_values"
new_dim_coor = pd.read_csv(path)
# load the new x,y values to the netCDF data
x = new_dim_coor["x"].dropna().astype("int").values
y = new_dim_coor["y"].values

In [5]:
tsnc_data_path = f"/uba/anemos_winddata/20191029_anemosDataFull/UBA-Windatlas/TSNC-Format/rho.10L.{year}.ts.nc"

data = xr.open_dataset(tsnc_data_path, engine='h5netcdf', chunks={"time":10000})
data = data.assign_coords(
                coords={"x": x,"y": y}
            )
data = data.reset_coords(names=["lat","lon"], drop=True)

def interpolate(xarray):
    #dummy = xarray.reset_coords(names=["lat","lon"], drop=True)
    dummy = xarray.interp(
                x=x_interp, 
                y=y_interp,
                level=level, 
                method="linear")

    return dummy.rho.load()

#interpolate(data).load()

In [6]:
template = data.rho.isel(x=[1], y=[1], level=[1])
template

<xarray.DataArray 'rho' (y: 1, x: 1, level: 1, time: 52560)>
dask.array<getitem, shape=(1, 1, 1, 52560), dtype=float32, chunksize=(1, 1, 1, 10000), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2017-01-01 ... 2017-12-31T23:50:00
  * x        (x) int64 -366000
  * y        (y) int64 -405000
  * level    (level) float32 60.0
Attributes:
    long_name:     4xDaily air density
    units:         kg/m3
    valid_range:   [0. 3.]
    var_desc:      Air Density
    actual_range:  [0.93370383 1.37516778]